# Extend kernel with additive noise for file ID

This notebook explores how to model the observation that experiments performed on different days, potentially prepared by different operators, lead to systematic differences in the measurable output

## Approach

The general idea is to consider a file-specific additive noise variable. To start thinking about this, consider data in file $j$ which are described by 
$$y_j = f_1(x_j) + \delta_j + \epsilon$$
where 
- $f_1\sim\mathcal{GP}(0, k_{x,x}(\theta_1))$ is the underlying process with covariance parameters $\theta_1$, 
- $\delta_j\sim\mathcal{N}(0,\sigma_f^2)$ is file-specific noise, and 
- $\epsilon\sim\mathcal{N}(0, \sigma^2)$ is the measurement noise.

One way to handle this conveniently is to consider a new input vector $u$ which is a one-hot encoding of the file ID $j$. Then we can write $\delta_j = f_2(u)$, with 
$$f_2\sim\mathcal{GP}(0, k_{u,u}(\theta_2))$$
where $\theta_2$ parameterises the covariance function of the file-specific noise.

Overall, we end up with a standard GP structure $y = f(x,u) + \epsilon$ where 
$$f(x,u) = f_1(x) + f_2(u)$$

### Log-scale

The problem with the approach above is that we know from experience that the output data are described better in log-space. Typically, we model as 
$$\log_{10}(y_j) = f_1(x_j) + \epsilon$$
But this is incompatible with the approach above, as adding a second kernel $f_2$ in the log-space is equivalent to multiplication in the real space. What we really want is
$$y_j = \exp\{f_1(x_j)\} + f_2(u) + \epsilon$$

## Load dependencies

In [ ]:
import os
import argparse
import pandas as pd
from pathlib import Path
import GPy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.insert( 0, '../')
sys.path.insert( 0, '../abex')
sys.path.insert( 0, '../pyBCKG')
sys.path.insert( 0, '../emukit')

import settings
from data import Dataset, onehot
import optim
import plotting

from bayesopt import BayesOptModel, flatten_list

## Fake data example

In [ ]:
def real_function(x):
    return x[:,0]*x[:,1] - 0.5*x[:,0] + 0.3*x[:,1] + 1.5

sigf = 1
sigy = 0.02

ns = 40
nf = 7
js = sigf*np.random.rand(nf,1)
xs = [np.random.rand(ns,2) for f in range(nf)]
X = np.concatenate(xs, axis=0)
print(X.shape)
F = np.vstack([f * np.ones((ns,1)) for f in range(nf)])
y = list(map(real_function, xs))
ys = np.concatenate([yi + js[f] for f,yi in enumerate(y)], axis=0)[:,np.newaxis]
print(ys.shape)
y = np.reshape(y, (ns*nf,1))
print(y.shape)
Y = ys + sigy*np.random.randn(ns*nf,1)

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter3D(X[:,0],X[:,1],y[:,0])
ax2 = fig.add_subplot(132)
ax2.scatter(y[:,0], ys[:,0])
ax3 = fig.add_subplot(133)
ax3.scatter(y[:,0], Y[:,0])

plt.tight_layout()
sns.despine()

In [ ]:
df = pd.DataFrame(np.hstack([X, F, y, ys, Y]), columns=['x0','x1','f','y (noise-free)','y (+file noise)','y (both noise)'])
df

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(9,4))
sns.scatterplot(x='x0', y='y (both noise)', hue='f', data=df, ax=axs[0])
sns.scatterplot(x='x1', y='y (both noise)', hue='f', data=df, ax=axs[1])
sns.despine()

### Train model without file information

In [ ]:
k0 = GPy.kern.Matern52(2)
model0 = GPy.models.GPRegression(X, Y, k0)
model0.optimize()
model0

In [ ]:
m0_mean, m0_var = model0.predict(X)
rs = np.corrcoef(Y.squeeze(), m0_mean.squeeze())
r = rs[1,0]

In [ ]:
f, axs = plt.subplots(2, 2, figsize=(10,10))

model0.plot(ax=axs[0,0], fixed_inputs=[(1,0.5)])
axs[0,0].set_xlabel('x0')
axs[0,0].set_ylabel('y')
model0.plot(ax=axs[0,1], fixed_inputs=[(0,0.5)])
axs[0,1].set_xlabel('x1')
axs[0,1].set_ylabel('y')
model0.plot(ax=axs[1,0])
axs[1,0].set_xlabel('x0')
axs[1,0].set_ylabel('x1')

axs[1,1].errorbar(Y.squeeze(), m0_mean.squeeze(), np.sqrt(m0_var.squeeze()), fmt='.')
axs[1,1].set_title('r = %1.2f' % r)
axs[1,1].set_xlabel('Data')
axs[1,1].set_ylabel('Model')

sns.despine()

### Train model with file information

In [ ]:
U = np.concatenate([np.tile(onehot(nf,f),(ns,1)) for f in range(nf)])

In [ ]:
XU = np.hstack([X,U])
for f in range(nf):
    plt.plot(range(ns*nf), U[:,f], '.')

In [ ]:
k1 = GPy.kern.Matern52(nf+2, active_dims=np.append(np.ones(2), np.zeros(nf)))
#k2 = GPy.kern.RBF(nf, active_dims=range(2,nf+2))
#k2 = GPy.kern.RBF(nf, active_dims=range(2,nf+2), ARD=True)
k2 = GPy.kern.Linear(nf, active_dims=range(2,nf+2))
combined_kernel = k1 + k2

In [ ]:
model1 = GPy.models.GPRegression(XU, Y, combined_kernel)
model1.optimize()
model1

In [ ]:
m1_mean, m1_var = model1.predict(XU)
rs1 = np.corrcoef(Y.squeeze(), m1_mean.squeeze())
r1 = rs1[1,0]

In [ ]:
f, axs = plt.subplots(2, 2, figsize=(10,10))

no_noise = [(i,0) for i in range(2,nf+2)]
model1.plot(ax=axs[0,0], fixed_inputs=[(1,0.5)]+no_noise)
axs[0,0].set_xlabel('x0')
axs[0,0].set_ylabel('y')
model1.plot(ax=axs[0,1], fixed_inputs=[(0,0.5)]+no_noise)
axs[0,1].set_xlabel('x1')
axs[0,1].set_ylabel('y')
model1.plot(ax=axs[1,0], fixed_inputs=no_noise)
axs[1,0].set_xlabel('x0')
axs[1,0].set_ylabel('x1')

axs[1,1].errorbar(Y.squeeze(), m1_mean.squeeze(), np.sqrt(m1_var.squeeze()), fmt='.')
axs[1,1].set_title('r = %1.2f' % r1)
axs[1,1].set_xlabel('Data')
axs[1,1].set_ylabel('Model')

sns.despine()

Comparison of the underlying real function and the inferred function

In [ ]:
ng = 21
xg = np.linspace(0,1,ng)
x1, x2 = np.meshgrid(xg, xg)
yreal = [real_function(np.hstack([xi*np.ones((ng,1)),xg[:,np.newaxis]])) for xi in xg]
ygp0_mean, ygp0_var = zip(*[model0.predict(np.hstack([xi*np.ones((ng,1)),xg[:,np.newaxis]])) for xi in xg])
ygp1_mean, ygp1_var = zip(*[model1.predict(np.hstack([xi*np.ones((ng,1)),xg[:,np.newaxis],np.zeros((ng,nf))])) for xi in xg])

In [ ]:
ygp0 = np.array(ygp0_mean).squeeze()
ygp1 = np.array(ygp1_mean).squeeze()

plt.pcolormesh(x1, x2, yreal, shading='auto', vmin=1, vmax=2.5)
plt.colorbar()

# Estimate the bias from the file noise
bias = js.mean()

f, axs = plt.subplots(1,3,figsize=(8,4))
p0 = axs[0].pcolormesh(x1, x2, np.abs(ygp0-yreal-bias), shading='auto', vmin=0.0, vmax=0.2, cmap='jet')
axs[0].set_title('Simple model: $y = f_1(x)$')
axs[1].pcolormesh(x1, x2, np.abs(ygp1-yreal-bias), shading='auto', vmin=0.0, vmax=0.2, cmap='jet')
axs[1].set_title('File-noise model: $y = f_1(x) + f_2(u)$')
axs[0].set_position([0.1, 0.1, 0.4, 0.8])
axs[1].set_position([0.58, 0.1, 0.4, 0.8])
axs[2].set_position([1, 0.1, 0.01, 0.8])
plt.colorbar(p0, cax=axs[2], label = '$| f_{real} - f_1 - b |$')

In [ ]:
ygp0 = np.sqrt(np.array(ygp0_var).squeeze())
ygp1 = np.sqrt(np.array(ygp1_var).squeeze())

f, axs = plt.subplots(1,4,figsize=(9,4))
p0 = axs[0].pcolormesh(x1, x2, ygp0, shading='auto')
axs[0].set_title('Simple model: $y = f_1(x)$')
p1 = axs[2].pcolormesh(x1, x2, ygp1, shading='auto')
axs[2].set_title('File-noise model: $y = f_1(x) + f_2(u)$')
axs[0].set_position([0., 0.1, 0.35, 0.8])
axs[1].set_position([0.38, 0.1, 0.01, 0.8])
plt.colorbar(p0, cax=axs[1])
axs[2].set_position([0.5, 0.1, 0.35, 0.8])
axs[3].set_position([0.88, 0.1, 0.01, 0.8])
plt.colorbar(p1, cax=axs[3])

### Conclusion

It looks like the additive file noise model leads to a better reconstruction of the original model, but with a constant bias. Ignoring the file noise leads to a model with slightly the wrong shape. 

The biggest effect is that including the file-noise enables much better accounting of uncertainty.